# Lab: การ Inference ของ LLM โดยใช้ PyTorch Transformers

โน้ตบุ๊กนี้จะแสดงการฝึกและทำ Inference ของ LLM โดยใช้ PyTorch/Transformers:
- **Eager Attention**: การทำงานแบบ attention ที่ไม่ได้ปรับแต่ง

โดยจะวัด:
- เวลาในการโหลดโมเดลและหน่วยความจำ
- ความเร็วและ throughput ของ inference
- ความเร็วในการ training (1 epoch บนชุดข้อมูลขนาดเล็ก)
- การใช้หน่วยความจำระหว่างการ training

**โมเดล**: Gemma 2B

---

## Hugging Face Token

ในการเข้าถึงโมเดล Gemma จะต้องยืนยันตัวตนกับ Hugging Face โปรดรับ read access token จาก [Hugging Face](https://huggingface.co/settings/tokens) และเพิ่มลงใน Colab secrets manager ตั้งชื่อ secret ว่า `HF_TOKEN`

หรือ Run Cell ต่อไปนี้เพื่อ Login

In [1]:
from huggingface_hub import notebook_login

notebook_login()

* * *

## เชื่อมต่อกับ T4 GPU

สามารถเชื่อมต่อกับ T4 GPU runtime ไปที่ "**Runtime/Connect (มุมบนขวา)**" -> "**Change runtime type**" และเลือก "**T4 GPU**" ภายใต้ "**Hardware accelerator**"

ตรวจสอบ GPU

In [2]:
!nvidia-smi

Fri Nov  7 05:11:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

* * *
## การติดตั้ง Library

In [3]:
%%capture
# ติดตั้งแพ็คเกจที่จำเป็น
!pip install -q transformers accelerate bitsandbytes peft datasets trl scipy

In [4]:
import torch
import time
import gc
import numpy as np
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    TextStreamer
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

# ตรวจสอบความพร้อมของ GPU
print(f"CUDA พร้อมใช้งาน: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_props = torch.cuda.get_device_properties(0)
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory(หน่วยความจำ): {gpu_props.total_memory / 1e9:.2f} GB")
    print(f"เวอร์ชัน CUDA: {torch.version.cuda}")

CUDA พร้อมใช้งาน: True
GPU: Tesla T4
GPU Memory(หน่วยความจำ): 15.83 GB
เวอร์ชัน CUDA: 12.6


## การตั้งค่า

In [5]:
# การตั้งค่าโมเดล
MODEL_NAME = "google/gemma-2b-it"

NUM_TRAIN_SAMPLES = 500
NUM_TRAIN_EPOCHS = 1
BATCH_SIZE = 2
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 2e-4

# prompts สำหรับทดสอบ inference
TEST_PROMPTS = [
    "อธิบาย quantum computing ให้เข้าใจง่ายๆ:",
    "ลำดับฟีโบนักชีคืออะไร?",
    "เขียนฟังก์ชัน Python เพื่อกลับข้อความ:"
]

## ฟังก์ชันเสริม

In [6]:
def get_gpu_memory_gb():
    """ดึงข้อมูลการใช้หน่วยความจำ GPU ปัจจุบันเป็น GB"""
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1e9
    return 0

def get_peak_memory_gb():
    """ดึงข้อมูลการใช้หน่วยความจำ GPU สูงสุดเป็น GB"""
    if torch.cuda.is_available():
        return torch.cuda.max_memory_allocated() / 1e9
    return 0

def clear_memory():
    """ล้างหน่วยความจำ GPU และระบบ"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

def format_time(seconds):
    """จัดรูปแบบวินาทีให้อ่านได้ง่าย"""
    return f"{seconds:.2f}วินาที ({seconds/60:.2f}นาที)"

def print_section(title):
    """พิมพ์ส่วนหัวที่จัดรูปแบบแล้ว"""
    print(f"\n{'='*80}")
    print(f"{title.center(80)}")
    print(f"{'='*80}\n")

## โหลดชุดข้อมูล

In [7]:
print("กำลังโหลดชุดข้อมูล instruction-tuned...")
dataset = load_dataset("mlabonne/FineTome-100k", split=f"train[:{NUM_TRAIN_SAMPLES}]")

def format_dataset(example):
    """จัดรูปแบบ conversations เป็นคู่ instruction-response"""
    # แยก conversation
    conversations = example.get("conversations", [])
    if len(conversations) < 2:
        return {"text": ""}

    # ดึง instruction ของผู้ใช้และ response ของ assistant
    user_msg = conversations[0].get("value", "")
    assistant_msg = conversations[1].get("value", "")

    # จัดรูปแบบเป็น instruction-response
    text = f"ผู้ใช้: {user_msg}\n\nผู้ช่วย: {assistant_msg}"
    return {"text": text}

dataset = dataset.map(format_dataset, remove_columns=dataset.column_names)
# กรองตัวอย่างที่ว่างออก
dataset = dataset.filter(lambda x: len(x["text"]) > 0)

print(f"โหลดชุดข้อมูลแล้ว: {len(dataset)} ตัวอย่าง")
print(f"ตัวอย่าง:\n{dataset[0]['text'][:200]}...")

กำลังโหลดชุดข้อมูล instruction-tuned...
โหลดชุดข้อมูลแล้ว: 500 ตัวอย่าง
ตัวอย่าง:
ผู้ใช้: Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of...


---

# ขั้นตอนที่ 1: การโหลดโมเดล

โหลด Gemma 2B ด้วย **eager attention**

In [8]:
print_section("กำลังโหลดโมเดลด้วย Eager Attention")

clear_memory()
start_time = time.time()
mem_before = get_gpu_memory_gb()


model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto",
    attn_implementation="eager"
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

load_time = time.time() - start_time
mem_after = get_gpu_memory_gb()
memory_used = mem_after - mem_before

print(f"  โหลดโมเดลเรียบร้อยแล้ว")
print(f"  การทำงานของ Attention: eager")
print(f"  เวลาในการโหลด: {format_time(load_time)}")
print(f"  หน่วยความจำที่ใช้: {memory_used:.3f} GB")
print(f"  หน่วยความจำ GPU ปัจจุบัน: {mem_after:.3f} GB")


                       กำลังโหลดโมเดลด้วย Eager Attention                       



`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  โหลดโมเดลเรียบร้อยแล้ว
  การทำงานของ Attention: eager
  เวลาในการโหลด: 27.36วินาที (0.46นาที)
  หน่วยความจำที่ใช้: 5.012 GB
  หน่วยความจำ GPU ปัจจุบัน: 5.012 GB


---

# ขั้นตอนที่ 2: การทดสอบ Inference

ทดสอบความเร็วของ inference ด้วย eager attention

In [9]:
print_section("การทดสอบ Inference - Eager Attention")

clear_memory()
times = []
outputs = []


inputs = tokenizer(TEST_PROMPTS[0], return_tensors="pt").to(model.device)
with torch.no_grad():
    _ = model.generate(**inputs, max_new_tokens=50, do_sample=False)

clear_memory()
mem_before = get_gpu_memory_gb()

print(f"\nกำลังรัน inference บน {len(TEST_PROMPTS)} prompts...\n")
for i, prompt in enumerate(TEST_PROMPTS, 1):
    print(f"Prompt {i}/{len(TEST_PROMPTS)}: {prompt[:50]}...")
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    start = time.time()
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=50, do_sample=False)
    elapsed = time.time() - start
    times.append(elapsed)

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    outputs.append(decoded)
    print(f"  เวลา: {elapsed:.4f}วินาที")

mem_after = get_gpu_memory_gb()
peak_mem = get_peak_memory_gb()

# ผลลัพธ์
avg_time = np.mean(times)
std_time = np.std(times)
tokens_per_sec = 50 / avg_time

print(f"\n{'='*60}")
print("ผลลัพธ์ INFERENCE (Eager Attention)")
print(f"{'='*60}")
print(f"เวลาเฉลี่ยต่อ Prompt: {avg_time:.4f}วินาที (±{std_time:.4f}วินาที)")
print(f"Tokens/วินาที: {tokens_per_sec:.2f}")
print(f"หน่วยความจำสูงสุด: {peak_mem:.3f} GB")
print(f"{'='*60}")

# เก็บผลลัพธ์
inference_results = {
    'avg_time': avg_time,
    'std_time': std_time,
    'tokens_per_sec': tokens_per_sec,
    'peak_memory': peak_mem,
    'outputs': outputs
}


                      การทดสอบ Inference - Eager Attention                      


กำลังรัน inference บน 3 prompts...

Prompt 1/3: อธิบาย quantum computing ให้เข้าใจง่ายๆ:...
  เวลา: 3.0521วินาที
Prompt 2/3: ลำดับฟีโบนักชีคืออะไร?...
  เวลา: 1.5675วินาที
Prompt 3/3: เขียนฟังก์ชัน Python เพื่อกลับข้อความ:...
  เวลา: 1.5492วินาที

ผลลัพธ์ INFERENCE (Eager Attention)
เวลาเฉลี่ยต่อ Prompt: 2.0563วินาที (±0.7042วินาที)
Tokens/วินาที: 24.32
หน่วยความจำสูงสุด: 5.025 GB


## ตัวอย่าง Outputs

In [10]:
print("\nตัวอย่าง Inference Outputs:\n")
for i, (prompt, output) in enumerate(zip(TEST_PROMPTS, outputs), 1):
    print(f"{'='*80}")
    print(f"Prompt {i}: {prompt}")
    print(f"{'-'*80}")
    print(f"Output: {output[:200]}...")
    print()


ตัวอย่าง Inference Outputs:

Prompt 1: อธิบาย quantum computing ให้เข้าใจง่ายๆ:
--------------------------------------------------------------------------------
Output: อธิบาย quantum computing ให้เข้าใจง่ายๆ:

**Quantum computing** คือการใช้เทคโนโลยีใหม่เพื่อสร้างระบบคอมพิวเตอร์ที่สามารถแก้ปัญหาที่ซับซ้อนได้อย่างรวดเร็ว เมื่อเทคนิคการคอมพิวเตอร์แบบด...

Prompt 2: ลำดับฟีโบนักชีคืออะไร?
--------------------------------------------------------------------------------
Output: ลำดับฟีโบนักชีคืออะไร?

1. ฟีโบนักชีพ
2. ฟีโบนักพืช
3. ฟีโบนักสัตว์
4. ฟีโบนักชีพและฟีโบนักพืช...

Prompt 3: เขียนฟังก์ชัน Python เพื่อกลับข้อความ:
--------------------------------------------------------------------------------
Output: เขียนฟังก์ชัน Python เพื่อกลับข้อความ:

```python
def reverse_string(string):
  """
  Reverses the order of the characters in a string.

  Args:
    string: The string to reverse.

  Returns:
    The ...



---

# ขั้นตอนที่ 3: การทดสอบการ training

Fine-tuning ด้วย LoRA เป็นเวลา 1 epoch เพื่อวัดประสิทธิภาพ training

In [11]:
print_section("การทดสอบ training - Eager Attention")

clear_memory()

# เพิ่ม LoRA adapters
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model_lora = get_peft_model(model, lora_config)
model_lora.print_trainable_parameters()


training_args = TrainingArguments(
    output_dir="./output_baseline",
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=10,
    save_strategy="no",
    report_to="none"
)

# สร้าง trainer
trainer = SFTTrainer(
    model=model_lora,
    args=training_args,
    train_dataset=dataset,
)

print(f"\nเริ่มการ training {len(dataset)} ตัวอย่างเป็นเวลา {NUM_TRAIN_EPOCHS} epoch...\n")

mem_before = get_gpu_memory_gb()
start_time = time.time()

train_result = trainer.train()

train_time = time.time() - start_time
mem_after = get_gpu_memory_gb()
peak_mem = get_peak_memory_gb()

# ผลลัพธ์
print(f"\n{'='*60}")
print("ผลลัพธ์การ traing (Eager Attention)")
print(f"{'='*60}")
print(f"เวลาในการ training: {format_time(train_time)}")
print(f"ตัวอย่าง/วินาที: {train_result.metrics['train_samples_per_second']:.2f}")
print(f"หน่วยความจำสูงสุด: {peak_mem:.3f} GB")
print(f"Loss สุดท้าย: {train_result.metrics['train_loss']:.4f}")
print(f"{'='*60}")

# เก็บผลลัพธ์
training_results = {
    'train_time': train_time,
    'samples_per_sec': train_result.metrics['train_samples_per_second'],
    'peak_memory': peak_mem,
    'final_loss': train_result.metrics['train_loss']
}


                      การทดสอบ training - Eager Attention                       

trainable params: 3,686,400 || all params: 2,509,858,816 || trainable%: 0.1469


The model is already on multiple devices. Skipping the move to device specified in `args`.



เริ่มการ training 500 ตัวอย่างเป็นเวลา 1 epoch...



OutOfMemoryError: CUDA out of memory. Tried to allocate 1.95 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.22 GiB is free. Process 57876 has 13.52 GiB memory in use. Of the allocated memory 12.59 GiB is allocated by PyTorch, and 816.27 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

---

# สรุป: ประสิทธิภาพ Baseline (Eager Attention)

ผลการทดสอบแบบสมบูรณ์โดยใช้ PyTorch มาตรฐานกับ eager attention

In [ ]:
print_section("สรุปประสิทธิภาพ BASELINE")

summary_data = {
    'เมตริก': [
        'เวลาโหลดโมเดล',
        'หน่วยความจำโหลดโมเดล',
        'เวลาเฉลี่ย Inference',
        'Tokens/วินาที Inference',
        'หน่วยความจำสูงสุด Inference',
        'เวลาการฝึก',
        'ตัวอย่าง/วินาที training',
        'หน่วยความจำสูงสุด training',
        'Loss สุดท้ายจากการ training'
    ],
    'ค่า': [
        format_time(load_time),
        f"{memory_used:.3f} GB",
        f"{inference_results['avg_time']:.4f}วินาที",
        f"{inference_results['tokens_per_sec']:.2f}",
        f"{inference_results['peak_memory']:.3f} GB",
        format_time(training_results['train_time']),
        f"{training_results['samples_per_sec']:.2f}",
        f"{training_results['peak_memory']:.3f} GB",
        f"{training_results['final_loss']:.4f}"
    ]
}

df = pd.DataFrame(summary_data)
print(df.to_string(index=False))

print(f"\n{'='*60}")
print("📝 จุดสำคัญ:")
print(f"{'='*60}")
print("- ใช้ EAGER attention")
print("- โมเดล: Gemma 2B (FP16)")
print(f"- การฝึก: {NUM_TRAIN_SAMPLES} ตัวอย่าง, {NUM_TRAIN_EPOCHS} epoch")
print("- LoRA fine-tuning (r=16)")